In [70]:
#Disable warning of Ripper implementation
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn import tree, svm
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn_evaluation import plot
import seaborn as sns
import time

In [71]:
data = arff.loadarff('ionosphere.arff')
df_iono = pd.DataFrame(data[0])

data = arff.loadarff('diabetes.arff')
df_diabe = pd.DataFrame(data[0])

data = arff.loadarff('vehicle.arff')
df_Vehicle = pd.DataFrame(data[0])

data = arff.loadarff('vowel.arff')
df_vowel = pd.DataFrame(data[0])

data = arff.loadarff('iris.arff')
df_iris = pd.DataFrame(data[0])

data = arff.loadarff('letter.arff')
df_letter = pd.DataFrame(data[0])

In [72]:
# Changing the last categorical class value into a numerical value
df_iono['class'] = pd.factorize(df_iono['class'])[0]

# Changing the last categorical class value into a numerical value
df_diabe['class'] = pd.factorize(df_diabe['class'])[0]

# Changing the last categorical class value into a numerical value
df_Vehicle['Class'] = pd.factorize(df_Vehicle['Class'])[0]

# Changing the last categorical class value into a numerical value
df_vowel['Class'] = pd.factorize(df_vowel['Class'])[0]

# Changing the last categorical class value into a numerical value
df_iris['class'] = pd.factorize(df_iris['class'])[0]

# Changing the last categorical class value into a numerical value
df_letter['class'] = pd.factorize(df_letter['class'])[0]


In [73]:
trainIono, testIono = train_test_split(df_iono, test_size=.25)
X_trainIono = trainIono.drop('class', axis=1)
y_trainIono = trainIono['class']
X_testIono = testIono.drop('class', axis=1)
y_testIono = testIono['class']

trainDiabe, testDiabe = train_test_split(df_diabe, test_size=.25)
X_trainDiabe = trainDiabe.drop('class', axis=1)
y_trainDiabe = trainDiabe['class']
X_testDiabe = testDiabe.drop('class', axis=1)
y_testDiabe = testDiabe['class']

trainVehicle, testVehicle = train_test_split(df_Vehicle, test_size=.25)
X_trainVehicle = trainVehicle.drop('Class', axis=1)
y_trainVehicle = trainVehicle['Class']
X_testVehicle = testVehicle.drop('Class', axis=1)
y_testVehicle = testVehicle['Class']

trainVowel, testVowel = train_test_split(df_vowel, test_size=.25)
X_trainVowel = trainVowel.drop('Class', axis=1)
y_trainVowel = trainVowel['Class']
X_testVowel = testVowel.drop('Class', axis=1)
y_testVowel = testVowel['Class']

trainIris, testIris = train_test_split(df_iris, test_size=.25)
X_trainIris = trainIris.drop('class', axis=1)
y_trainIris = trainIris['class']
X_testIris = testIris.drop('class', axis=1)
y_testIris = testIris['class']

trainLetter, testLetter = train_test_split(df_letter, test_size=.25)
X_trainLetter = trainLetter.drop('class', axis=1)
y_trainLetter = trainLetter['class']
X_testLetter = testLetter.drop('class', axis=1)
y_testLetter = testLetter['class']

In [74]:
data = [
    ('Ionosphere', X_trainIono, y_trainIono, X_testIono, y_testIono),
    ('Diabetes', X_trainDiabe, y_trainDiabe,X_testDiabe, y_testDiabe),
    ('Vehicle', X_trainVehicle, y_trainVehicle, X_testVehicle, y_testVehicle),
    ('Vowel', X_trainVowel, y_trainVowel, X_testVowel, y_testVowel),
    ('Iris', X_trainIris, y_trainIris, X_testIris, y_testIris)
]

# Comparison without using GridSearch

In [75]:
clf_tree = tree.DecisionTreeClassifier()
clf_svm = svm.SVC()
clf_BaggingTree = BaggingClassifier(estimator=clf_tree)
clf_AdaBoostSAMMETree = AdaBoostClassifier(estimator=clf_tree,algorithm='SAMME')
clf_AdaBoostSAMMERTree = AdaBoostClassifier(estimator=clf_tree,algorithm='SAMME.R')
clf_BaggingSVM = BaggingClassifier(estimator=clf_svm)
clf_AdaBoostSAMMESVM = AdaBoostClassifier(estimator=clf_svm,algorithm='SAMME')
clf_AdaBoostSAMMERSVM = AdaBoostClassifier(estimator=clf_svm,algorithm='SAMME.R')
clf_GradBoost = GradientBoostingClassifier()

In [76]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreSVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_svm,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreSVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.9314814814814815 +- 0.03770195387504116
Tiempo medio en ejecutarse el método (train): 0.004579234123229981 +- 0.004579234123229981s
Tiempo medio en ejecutarse el método (score): 0.0017377376556396485 +- 0.0017377376556396485s
Tiempo en ejecutarse la búsqueda 0.9337513446807861s, (0.015562522411346435 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7640048396854204 +- 0.035742068017913776
Tiempo medio en ejecutarse el método (train): 0.006525444984436035 +- 0.006525444984436035s
Tiempo medio en ejecutarse el método (score): 0.0023399829864501954 +- 0.0023399829864501954s
Tiempo en ejecutarse la búsqueda 0.3692481517791748s, (0.0061541358629862465 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.774454365079365 +- 0.0067896581874082144
Tiempo medio en ejecutarse el método (train): 0.008519721031188966 +- 0.008519721031188966s
Tiempo medio en ejecutars

In [77]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreTree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_tree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreTree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreTree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8374643874643874 +- 0.07729006580069052
Tiempo medio en ejecutarse el método (train): 0.004087233543395996 +- 0.004087233543395996s
Tiempo medio en ejecutarse el método (score): 0.0011581182479858398 +- 0.0011581182479858398s
Tiempo en ejecutarse la búsqueda 0.02764606475830078s, (0.00046076774597167967 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7101935874168179 +- 0.04500200988278143
Tiempo medio en ejecutarse el método (train): 0.002818870544433594 +- 0.002818870544433594s
Tiempo medio en ejecutarse el método (score): 0.0009518146514892578 +- 0.0009518146514892578s
Tiempo en ejecutarse la búsqueda 0.0243380069732666s, (0.00040563344955444334 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9274553571428571 +- 0.02247747756903065
Tiempo medio en ejecutarse el método (train): 0.0031865596771240234 +- 0.0031865596771240234s
Tiempo medio en ejecu

In [78]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBaggingSVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_BaggingSVM,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBaggingSVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBaggingSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.9390313390313392 +- 0.035019564757840814
Tiempo medio en ejecutarse el método (train): 0.017972660064697266 +- 0.017972660064697266s
Tiempo medio en ejecutarse el método (score): 0.00326082706451416 +- 0.00326082706451416s
Tiempo en ejecutarse la búsqueda 0.04097485542297363s, (0.0006829142570495605 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7553236539624923 +- 0.03746984277115929
Tiempo medio en ejecutarse el método (train): 0.03503282070159912 +- 0.03503282070159912s
Tiempo medio en ejecutarse el método (score): 0.006067657470703125 +- 0.006067657470703125s
Tiempo en ejecutarse la búsqueda 0.06349635124206543s, (0.0010582725207010904 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.782266865079365 +- 0.01623905590752624
Tiempo medio en ejecutarse el método (train): 0.03530642986297607 +- 0.03530642986297607s
Tiempo medio en ejecutarse el méto

In [79]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBaggingTree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_BaggingTree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBaggingTree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBaggingTree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8864672364672366 +- 0.05517088683946423
Tiempo medio en ejecutarse el método (train): 0.025995373725891113 +- 0.025995373725891113s
Tiempo medio en ejecutarse el método (score): 0.00195004940032959 +- 0.00195004940032959s
Tiempo en ejecutarse la búsqueda 0.04006671905517578s, (0.0006677786509195964 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7484875983061101 +- 0.058898794166957466
Tiempo medio en ejecutarse el método (train): 0.019966316223144532 +- 0.019966316223144532s
Tiempo medio en ejecutarse el método (score): 0.002074861526489258 +- 0.002074861526489258s
Tiempo en ejecutarse la búsqueda 0.029908418655395508s, (0.0004984736442565918 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9431795634920634 +- 0.026691728206154137
Tiempo medio en ejecutarse el método (train): 0.019844841957092286 +- 0.019844841957092286s
Tiempo medio en ejecutarse 

In [80]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting1_SVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMESVM,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting1_SVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting1_SVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.6273504273504272 +- 0.014613468493711852
Tiempo medio en ejecutarse el método (train): 0.18725643157958985 +- 0.18725643157958985s
Tiempo medio en ejecutarse el método (score): 0.013151288032531738 +- 0.013151288032531738s
Tiempo en ejecutarse la búsqueda 0.2835848331451416s, (0.00472641388575236 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.6475801572897761 +- 0.006795313546864396
Tiempo medio en ejecutarse el método (train): 0.22922468185424805 +- 0.22922468185424805s
Tiempo medio en ejecutarse el método (score): 0.013848662376403809 +- 0.013848662376403809s
Tiempo en ejecutarse la búsqueda 0.7374529838562012s, (0.012290883064270019 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.7760416666666666 +- 0.005379143536399193
Tiempo medio en ejecutarse el método (train): 0.6693656921386719 +- 0.6693656921386719s
Tiempo medio en ejecutarse el método (

In [81]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting1_Tree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMETree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting1_Tree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting1_Tree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.834045584045584 +- 0.09155511245900665
Tiempo medio en ejecutarse el método (train): 0.005106496810913086 +- 0.005106496810913086s
Tiempo medio en ejecutarse el método (score): 0.0011346101760864257 +- 0.0011346101760864257s
Tiempo en ejecutarse la búsqueda 0.015501022338867188s, (0.0002583503723144531 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7050211736237144 +- 0.04861825492292677
Tiempo medio en ejecutarse el método (train): 0.003986263275146484 +- 0.003986263275146484s
Tiempo medio en ejecutarse el método (score): 0.0009840965270996095 +- 0.0009840965270996095s
Tiempo en ejecutarse la búsqueda 0.012218713760375977s, (0.0002036452293395996 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9274553571428571 +- 0.021327129935489998
Tiempo medio en ejecutarse el método (train): 0.004055023193359375 +- 0.004055023193359375s
Tiempo medio en ejecut

AdaBoost with SVM and SAMMER_R is not supported

In [82]:
"""
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting2_SVM = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMERSVM,i[1], y=i[2],cv=10, n_jobs=-1, error_score='raise')
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting2_SVM.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=Score)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())
"""

'\nDatasets = []\nTimeTrain = []\nTimeScore = []\nTimeCV = []\nScoreBoosting2_SVM = []\nfor i in data:\n    init = time.time()\n    cv = cross_validate(clf_AdaBoostSAMMERSVM,i[1], y=i[2],cv=10, n_jobs=-1, error_score=\'raise\')\n    end = time.time()\n    timeCV = end - init\n    print(f"\n--------- {i[0]} ---------")\n    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv[\'test_score\'])} +- {np.std(cv[\'test_score\'])}")\n    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv[\'fit_time\'])} +- {np.mean(np.mean(cv[\'fit_time\']))}s")\n    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv[\'score_time\'])} +- {np.mean(np.mean(cv[\'score_time\']))}s")\n    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")\n    Datasets.append(i[0])\n    TimeTrain.append(np.mean(cv[\'fit_time\']))\n    TimeScore.append(np.mean(cv[\'score_time\']))\n    TimeCV.append(timeCV)\n    ScoreBoosting2_SVM.append(np.mean(cv[\'test_score\']))\n\nm

In [83]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreBoosting2_Tree = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_AdaBoostSAMMERTree,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreBoosting2_Tree.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreBoosting2_Tree)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8415954415954415 +- 0.08626835197996811
Tiempo medio en ejecutarse el método (train): 0.00485844612121582 +- 0.00485844612121582s
Tiempo medio en ejecutarse el método (score): 0.0011798381805419923 +- 0.0011798381805419923s
Tiempo en ejecutarse la búsqueda 0.014097929000854492s, (0.00023496548334757487 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7102238354506957 +- 0.04805688212871203
Tiempo medio en ejecutarse el método (train): 0.0039041996002197265 +- 0.0039041996002197265s
Tiempo medio en ejecutarse el método (score): 0.0011305809020996094 +- 0.0011305809020996094s
Tiempo en ejecutarse la búsqueda 0.01083827018737793s, (0.00018063783645629882 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.9321428571428572 +- 0.022488954677418717
Tiempo medio en ejecutarse el método (train): 0.003948593139648437 +- 0.003948593139648437s
Tiempo medio en ejec

In [84]:
Datasets = []
TimeTrain = []
TimeScore = []
TimeCV = []
ScoreGradBoost = []
for i in data:
    init = time.time()
    cv = cross_validate(clf_GradBoost,i[1], y=i[2],cv=10, n_jobs=-1)
    end = time.time()
    timeCV = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"Score de la Validación Cruzada:\n   score = {np.mean(cv['test_score'])} +- {np.std(cv['test_score'])}")
    print(f"Tiempo medio en ejecutarse el método (train): {np.mean(cv['fit_time'])} +- {np.mean(np.mean(cv['fit_time']))}s")
    print(f"Tiempo medio en ejecutarse el método (score): {np.mean(cv['score_time'])} +- {np.mean(np.mean(cv['score_time']))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeCV}s, ({timeCV/60} min)")
    Datasets.append(i[0])
    TimeTrain.append(np.mean(cv['fit_time']))
    TimeScore.append(np.mean(cv['score_time']))
    TimeCV.append(timeCV)
    ScoreGradBoost.append(np.mean(cv['test_score']))

my_dict = dict(Dataset=Datasets,TimeTrain=TimeTrain, TimeScore=TimeScore,TimeCV=TimeCV, Score=ScoreGradBoost)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
Score de la Validación Cruzada:
   score = 0.8941595441595442 +- 0.05901993229286047
Tiempo medio en ejecutarse el método (train): 0.14216749668121337 +- 0.14216749668121337s
Tiempo medio en ejecutarse el método (score): 0.0013455390930175782 +- 0.0013455390930175782s
Tiempo en ejecutarse la búsqueda 0.1622171401977539s, (0.0027036190032958983 min)

--------- Diabetes ---------
Score de la Validación Cruzada:
   score = 0.7762552934059286 +- 0.04166924398294609
Tiempo medio en ejecutarse el método (train): 0.0975724458694458 +- 0.0975724458694458s
Tiempo medio en ejecutarse el método (score): 0.0013717174530029296 +- 0.0013717174530029296s
Tiempo en ejecutarse la búsqueda 0.1145009994506836s, (0.0019083499908447265 min)

--------- Vehicle ---------
Score de la Validación Cruzada:
   score = 0.962078373015873 +- 0.02032533583819248
Tiempo medio en ejecutarse el método (train): 0.13828034400939943 +- 0.13828034400939943s
Tiempo medio en ejecutarse el métod

In [85]:
compareDF = pd.DataFrame()
compareDF['SVM'] = ScoreSVM
compareDF['Tree'] = ScoreTree
compareDF['BaggingSVM'] = ScoreBaggingSVM
compareDF['BaggingTree'] = ScoreBaggingTree
compareDF['Boosting1_SVM'] = ScoreBoosting1_SVM
compareDF['Boosting1_Tree'] = ScoreBoosting1_Tree
compareDF['Boosting2_Tree'] = ScoreBoosting2_Tree
compareDF['GradBoost'] = ScoreGradBoost
compareDF.to_csv("performance.csv", index=False)
print(compareDF)
print(compareDF.to_latex())

        SVM      Tree  BaggingSVM  BaggingTree  Boosting1_SVM  Boosting1_Tree  \
0  0.931481  0.837464    0.939031     0.886467       0.627350        0.834046   
1  0.764005  0.710194    0.755324     0.748488       0.647580        0.705021   
2  0.774454  0.927455    0.782267     0.943180       0.776042        0.927455   
3  0.963604  0.990541    0.964973     0.982432       0.909586        0.986505   
4  0.973485  0.928788    0.973485     0.946970       0.624242        0.920455   

   Boosting2_Tree  GradBoost  
0        0.841595   0.894160  
1        0.710224   0.776255  
2        0.932143   0.962078  
3        0.990541   0.989189  
4        0.920455   0.929545  
\begin{tabular}{lrrrrrrrr}
\toprule
{} &       SVM &      Tree &  BaggingSVM &  BaggingTree &  Boosting1\_SVM &  Boosting1\_Tree &  Boosting2\_Tree &  GradBoost \\
\midrule
0 &  0.931481 &  0.837464 &    0.939031 &     0.886467 &       0.627350 &        0.834046 &        0.841595 &   0.894160 \\
1 &  0.764005 &  0.710194 &   

In [103]:
import numpy as np
from scipy.stats import chi2

def compute_iman_davenport_statistic(performance_matrix):
  # Compute the ranks of the model performance on each dataset
  ranks = np.apply_along_axis(lambda x: len(x) - np.argsort(np.argsort(x)), 1, performance_matrix)

  # Sum the ranks for each model across all datasets
  rank_sums = np.sum(ranks, axis=0)

  # Compute the iman Davenport statistic
  iman_davenport_statistic = (np.max(rank_sums) - np.min(rank_sums)) / performance_matrix.shape[1]

  return iman_davenport_statistic

def compute_p_value(iman_davenport_statistic, num_models, num_datasets):
  # Compute the degrees of freedom for the iman Davenport test
  df = num_models - 1

  # Compute the p-value using the chi-squared distribution
  p_value = 1 - chi2.cdf(iman_davenport_statistic, df)

  return p_value

def iman_davenport_test(performance_matrix, significance_level):
  # Compute the iman Davenport statistic and p-value
  iman_davenport_statistic = compute_iman_davenport_statistic(performance_matrix)
  p_value = compute_p_value(iman_davenport_statistic, performance_matrix.shape[0], performance_matrix.shape[1])

  # Determine whether the difference in performance between the models is statistically significant
  if p_value < significance_level:
    print(f"The difference in performance between the models is statistically significant (p = {p_value:.3f})")
  else:
    print(f"The difference in performance between the models is not statistically significant (p = {p_value:.3f})")
  return p_value

In [104]:
performance_matrix = pd.read_csv("performance.csv")
# Run the iman Davenport test
p_value = iman_davenport_test(performance_matrix, 0.05)

The difference in performance between the models is not statistically significant (p = 0.497)


In [105]:
# If there are significant differences between the models, then apply the wilcoxon test to determine which models are significantly different
if p_value < 0.05:
  # Compute the pairwise differences between the models
  pairwise_differences = np.apply_along_axis(lambda x: x - x[:, None], 1, performance_matrix)

  # Compute the p-values for the pairwise differences using the Wilcoxon signed-rank test
  p_values = np.apply_along_axis(lambda x: wilcoxon(x, zero_method="wilcox")[1], 1, pairwise_differences)

  # Compute the Bonferroni correction
  bonferroni_correction = 0.05 / (p_values.shape[0] * (p_values.shape[0] - 1) / 2)

  # Determine which models are significantly different
  significant_differences = np.argwhere(p_values < bonferroni_correction)

  # Print the significant differences
  for difference in significant_differences:
    print(f"Model {difference[0]} is significantly different from model {difference[1]}")

# Comparison using GridSearch

In [89]:
parametersSVM = [
    {"kernel": ["linear"], "C": [1, 10]}
]
parametersTree = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter' : ['best', 'random']
}
parametersSVMBagging = [
    {
    'n_estimators':[10], # Numero de estimators = 10 porque si no tarda demasiado
    "estimator__kernel": ["rbf"],
    "estimator__C": [0.01, 0.1],
    'estimator__gamma': [0.01, 0.1],
    #'max_samples': [0.75, 1]
    # Saltan warnings si aleatoriamente solo seleccionamos instancias de una clase. Podríamos ignorar dichos fits o capturar los warning. Eliminamos el problema directamente
    'max_features': [0.5, 0.75],
    'bootstrap': [True, False]
    }
]
parametersTreeBagging = {
    'estimator__criterion':['gini', 'entropy', 'log_loss'],
    'estimator__splitter' : ['best', 'random'],
    # {'max_samples': [0.75, 1],
    'max_features': [0.5, 0.75, 1],
    'bootstrap': [True, False]
}
parametersSVMBoosting = [
    {
    "estimator__kernel": ["rbf"],
    "estimator__C": [1, 10, 100, 1000],
    'estimator__gamma': [0.01, 0.1, 1],
    'n_estimators':[25, 50],
    'learning_rate':[0.1, 1, 10]
    }
]
parametersTreeBoosting = {
    'estimator__criterion':['gini', 'entropy', 'log_loss'],
    'estimator__splitter' : ['best', 'random'],
    'n_estimators':[25, 50],
    'learning_rate':[0.1,1,10]
}
parametersGradBoosting = {
    "loss" : ['log_loss', 'deviance', 'exponential'],
    'learning_rate' : [0.01, 0.1, 1],
    'n_estimators': [100],
    'criterion': ['friedman_mse', 'squared_error'],
    'warm_start': [True, False]
}

optimalSVM = GridSearchCV(estimator=clf_svm, cv=10, param_grid=parametersSVM, n_jobs=-1)
optimalTree = GridSearchCV(estimator=clf_tree, cv=10, param_grid=parametersTree, n_jobs=-1)
optimalSVMBag = GridSearchCV(estimator=clf_BaggingSVM, cv=10, param_grid=parametersSVMBagging, n_jobs=-1)
optimalTreeBag = GridSearchCV(estimator=clf_BaggingTree, cv=10, param_grid=parametersTreeBagging, n_jobs=-1)
optimalSVMBoost1 = GridSearchCV(estimator=clf_AdaBoostSAMMESVM, cv=10, param_grid=parametersSVMBoosting, n_jobs=-1)
optimalTreeBoost1 = GridSearchCV(estimator=clf_AdaBoostSAMMETree, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalSVMBoost2 = GridSearchCV(estimator=clf_AdaBoostSAMMERSVM, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalTreeBoost2 = GridSearchCV(estimator=clf_AdaBoostSAMMERTree, cv=10, param_grid=parametersTreeBoosting, n_jobs=-1)
optimalGradBoost = GridSearchCV(estimator=clf_GradBoost, cv=10, param_grid=parametersGradBoosting, n_jobs=-1)

In [90]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreSVM = []
for i in data:
    init = time.time()
    optimalSVM.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVM:")
    print(f'    Best params -> {optimalSVM.best_params_}')
    print(f'    Best score -> {optimalSVM.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVM.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVM.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVM.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVM.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreSVM.append(optimalSVM.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreSVM)
SVMDF = pd.DataFrame (my_dict)
print(SVMDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 10, 'kernel': 'linear'}
    Best score -> 0.8665242165242164
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9204545454545454
Tiempo medio en ejecutarse el método: 0.0057098627090454105 +- 0.0015075883583829403s
Tiempo en ejecutarse la búsqueda 0.03313899040222168s, (0.0005523165067036947 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 10, 'kernel': 'linear'}
    Best score -> 0.7778584392014519
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7552083333333334
Tiempo medio en ejecutarse el método: 7.2714057803153995 +- 1.8135194165752377s
Tiempo en ejecutarse la búsqueda 35.120866537094116s, (0.5853477756182353 min)

--------- Vehicle ---------
La mejor accuracy se obtuvo con el siguiente SVM:
    Best params -> {'C': 1, 'kernel': 'linear'}
    Best score -> 0.9

In [91]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreTree = []
for i in data:
    init = time.time()
    optimalTree.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente Tree:")
    print(f'    Best params -> {optimalTree.best_params_}')
    print(f'    Best score -> {optimalTree.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTree.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTree.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTree.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTree.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreTree.append(optimalTree.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreTree)
TreeDF = pd.DataFrame (my_dict)
print(TreeDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterion': 'gini', 'splitter': 'random'}
    Best score -> 0.8824786324786326
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9204545454545454
Tiempo medio en ejecutarse el método: 0.003306011358896891 +- 0.0004200355114786722s
Tiempo en ejecutarse la búsqueda 0.04532051086425781s, (0.0007553418477376302 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterion': 'entropy', 'splitter': 'best'}
    Best score -> 0.7341500302480339
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7239583333333334
Tiempo medio en ejecutarse el método: 0.002582216262817383 +- 0.0002444373606118915s
Tiempo en ejecutarse la búsqueda 0.04037785530090332s, (0.0006729642550150554 min)

--------- Vehicle ---------
La mejor accuracy se obtuvo con el siguiente Tree:
    Best params -> {'criterio

In [92]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBaggingSVM = []
for i in data:
    init = time.time()
    optimalSVMBag.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBag:")
    print(f'    Best params -> {optimalSVMBag.best_params_}')
    print(f'    Best score -> {optimalSVMBag.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBag.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBag.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBag.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBag.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBaggingSVM.append(optimalSVMBag.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBaggingSVM)
SVMBagDF = pd.DataFrame (my_dict)
print(SVMBagDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVMBag:
    Best params -> {'bootstrap': False, 'estimator__C': 0.1, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'max_features': 0.75, 'n_estimators': 10}
    Best score -> 0.8750712250712251
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.8977272727272727
Tiempo medio en ejecutarse el método: 0.027958682179450987 +- 0.0030394936244046555s
Tiempo en ejecutarse la búsqueda 0.4098327159881592s, (0.00683054526646932 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVMBag:
    Best params -> {'bootstrap': True, 'estimator__C': 0.01, 'estimator__gamma': 0.01, 'estimator__kernel': 'rbf', 'max_features': 0.5, 'n_estimators': 10}
    Best score -> 0.6475801572897761
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.6614583333333334
Tiempo medio en ejecutarse el método: 0.08240163475275039 +- 0.008381181150688586s
Tiempo e

In [93]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBaggingTree = []
for i in data:
    init = time.time()
    optimalTreeBag.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBag:")
    print(f'    Best params -> {optimalTreeBag.best_params_}')
    print(f'    Best score -> {optimalTreeBag.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBag.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBag.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBag.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBag.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBaggingTree.append(optimalTreeBag.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBaggingTree)
TreeBagDF = pd.DataFrame (my_dict)
print(TreeBagDF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBag:
    Best params -> {'bootstrap': True, 'estimator__criterion': 'log_loss', 'estimator__splitter': 'best', 'max_features': 0.5}
    Best score -> 0.9353276353276353
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9204545454545454
Tiempo medio en ejecutarse el método: 0.020287468036015825 +- 0.0022619097959124014s
Tiempo en ejecutarse la búsqueda 0.701573371887207s, (0.01169288953145345 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBag:
    Best params -> {'bootstrap': True, 'estimator__criterion': 'log_loss', 'estimator__splitter': 'best', 'max_features': 0.75}
    Best score -> 0.7554446460980037
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7291666666666666
Tiempo medio en ejecutarse el método: 0.018671829170650907 +- 0.0019951253656198655s
Tiempo en ejecutarse la búsqueda 0.6113367080688477s, (0.01018

In [94]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting1_SVM = []
for i in data:
    init = time.time()
    optimalSVMBoost1.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost1:")
    print(f'    Best params -> {optimalSVMBoost1.best_params_}')
    print(f'    Best score -> {optimalSVMBoost1.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBoost1.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost1.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBoost1.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBoost1.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting1_SVM.append(optimalSVMBoost1.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting1_SVM)
SVMBoost1DF = pd.DataFrame (my_dict)
print(SVMBoost1DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente SVMBoost1:
    Best params -> {'estimator__C': 100, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'learning_rate': 0.1, 'n_estimators': 25}
    Best score -> 0.9465811965811965
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9659090909090909
Tiempo medio en ejecutarse el método: 0.1751982973681556 +- 0.030331102517580852s
Tiempo en ejecutarse la búsqueda 8.788516283035278s, (0.1464752713839213 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente SVMBoost1:
    Best params -> {'estimator__C': 1000, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'learning_rate': 0.1, 'n_estimators': 25}
    Best score -> 0.651028433151845
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.6614583333333334
Tiempo medio en ejecutarse el método: 0.6698947300513586 +- 0.1464919469735082s
Tiempo en ejecutarse la búsqueda 32.29699921607971

In [95]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting1_Tree = []
for i in data:
    init = time.time()
    optimalTreeBoost1.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBoost1:")
    print(f'    Best params -> {optimalTreeBoost1.best_params_}')
    print(f'    Best score -> {optimalTreeBoost1.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBoost1.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBoost1.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBoost1.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBoost1.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting1_Tree.append(optimalTreeBoost1.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting1_Tree)
TreeBoost1DF = pd.DataFrame (my_dict)
print(TreeBoost1DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost1:
    Best params -> {'estimator__criterion': 'log_loss', 'estimator__splitter': 'random', 'learning_rate': 10, 'n_estimators': 50}
    Best score -> 0.9054131054131055
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9318181818181818
Tiempo medio en ejecutarse el método: 0.004281731446584066 +- 0.0007377502838852982s
Tiempo en ejecutarse la búsqueda 0.20442605018615723s, (0.0034071008364359536 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost1:
    Best params -> {'estimator__criterion': 'entropy', 'estimator__splitter': 'best', 'learning_rate': 0.1, 'n_estimators': 25}
    Best score -> 0.7356624319419238
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7291666666666666
Tiempo medio en ejecutarse el método: 0.00347496403588189 +- 0.0003881414765801144s
Tiempo en ejecutarse la búsqueda 0.193420886993408

In [96]:
"""
Datasets = []
TimeSearch = []
TimeMethod = []
Score = []
for i in data:
    init = time.time()
    optimalSVMBoost2.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost2:")
    print(f'    Best params -> {optimalSVMBoost2.best_params_}')
    print(f'    Best score -> {optimalSVMBoost2.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalSVMBoost2.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost2.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalSVMBoost2.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalSVMBoost2.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    Score.append(optimalSVMBoost2.score(i[3], i[4]))

d7 = dict(SVM_Boost2=Score)
my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=Score)
SVMBoost2DF = pd.DataFrame (my_dict)
print(SVMBoost2DF.to_latex())
"""

'\nDatasets = []\nTimeSearch = []\nTimeMethod = []\nScore = []\nfor i in data:\n    init = time.time()\n    optimalSVMBoost2.fit(i[1], i[2])\n    end = time.time()\n    timeSearch = end - init\n    print(f"\n--------- {i[0]} ---------")\n    print(f"La mejor accuracy se obtuvo con el siguiente SVMBoost2:")\n    print(f\'    Best params -> {optimalSVMBoost2.best_params_}\')\n    print(f\'    Best score -> {optimalSVMBoost2.best_score_}\')\n\n    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")\n    print(f"    score = {optimalSVMBoost2.score(i[3], i[4])}")\n    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalSVMBoost2.cv_results_.get(\'mean_fit_time\'))} +- {np.mean(optimalSVMBoost2.cv_results_.get(\'std_fit_time\'))}s")\n    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")\n    Datasets.append(i[0])\n    TimeMethod.append(np.mean(optimalSVMBoost2.cv_results_.get(\'mean_fit_time\')))\n    TimeSearch.append(timeSearch)

In [97]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreBoosting2_Tree = []
for i in data:
    init = time.time()
    optimalTreeBoost2.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente TreeBoost2:")
    print(f'    Best params -> {optimalTreeBoost2.best_params_}')
    print(f'    Best score -> {optimalTreeBoost2.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalTreeBoost2.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalTreeBoost2.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalTreeBoost2.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalTreeBoost2.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreBoosting2_Tree.append(optimalTreeBoost2.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreBoosting2_Tree)
TreeBoost2DF = pd.DataFrame (my_dict)
print(TreeBoost2DF.to_latex())


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost2:
    Best params -> {'estimator__criterion': 'gini', 'estimator__splitter': 'random', 'learning_rate': 10, 'n_estimators': 25}
    Best score -> 0.8863247863247864
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.8522727272727273
Tiempo medio en ejecutarse el método: 0.004264355368084377 +- 0.0005574008033172223s
Tiempo en ejecutarse la búsqueda 0.20955538749694824s, (0.003492589791615804 min)

--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente TreeBoost2:
    Best params -> {'estimator__criterion': 'entropy', 'estimator__splitter': 'best', 'learning_rate': 10, 'n_estimators': 50}
    Best score -> 0.7304900181488204
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.734375
Tiempo medio en ejecutarse el método: 0.003472867939207289 +- 0.0004509414860219145s
Tiempo en ejecutarse la búsqueda 0.18791437149047852s, (0.0031319

In [98]:
Datasets = []
TimeSearch = []
TimeMethod = []
ScoreGradBoost = []
for i in data:
    init = time.time()
    optimalGradBoost.fit(i[1], i[2])
    end = time.time()
    timeSearch = end - init
    print(f"\n--------- {i[0]} ---------")
    print(f"La mejor accuracy se obtuvo con el siguiente GradBoost:")
    print(f'    Best params -> {optimalGradBoost.best_params_}')
    print(f'    Best score -> {optimalGradBoost.best_score_}')

    print(f"Si usamos el dataset de test, obtenemos el siguiente resultado:")
    print(f"    score = {optimalGradBoost.score(i[3], i[4])}")
    print(f"Tiempo medio en ejecutarse el método: {np.mean(optimalGradBoost.cv_results_.get('mean_fit_time'))} +- {np.mean(optimalGradBoost.cv_results_.get('std_fit_time'))}s")
    print(f"Tiempo en ejecutarse la búsqueda {timeSearch}s, ({timeSearch/60} min)")
    Datasets.append(i[0])
    TimeMethod.append(np.mean(optimalGradBoost.cv_results_.get('mean_fit_time')))
    TimeSearch.append(timeSearch)
    ScoreGradBoost.append(optimalGradBoost.score(i[3], i[4]))

my_dict = dict(Dataset=Datasets,TimeMethod=TimeMethod,TimeSearch=TimeSearch, Score=ScoreGradBoost)
GradBoostDF = pd.DataFrame (my_dict)
print(GradBoostDF.to_latex())

/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter nam


--------- Ionosphere ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 100, 'warm_start': True}
    Best score -> 0.9015669515669515
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9659090909090909
Tiempo medio en ejecutarse el método: 0.1482191589143541 +- 0.010765969189639327s
Tiempo en ejecutarse la búsqueda 3.5962140560150146s, (0.05993690093358358 min)


/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter nam


--------- Diabetes ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 100, 'warm_start': False}
    Best score -> 0.7762552934059286
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.7395833333333334
Tiempo medio en ejecutarse el método: 0.11180503103468155 +- 0.008481147430610105s
Tiempo en ejecutarse la búsqueda 2.726360559463501s, (0.045439342657725014 min)


/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter nam


--------- Vehicle ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'deviance', 'n_estimators': 100, 'warm_start': False}
    Best score -> 0.976314484126984
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9716981132075472
Tiempo medio en ejecutarse el método: 0.15736069679260256 +- 0.012347092363437926s
Tiempo en ejecutarse la búsqueda 3.815310001373291s, (0.06358850002288818 min)


/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter nam


--------- Vowel ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 100, 'warm_start': False}
    Best score -> 0.991891891891892
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9878542510121457
Tiempo medio en ejecutarse el método: 0.1740961015224457 +- 0.018555328069715667s
Tiempo en ejecutarse la búsqueda 4.208066701889038s, (0.07013444503148397 min)


/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/home/ricardo/PycharmProjects/Local_IMD/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter nam


--------- Iris ---------
La mejor accuracy se obtuvo con el siguiente GradBoost:
    Best params -> {'criterion': 'friedman_mse', 'learning_rate': 1, 'loss': 'log_loss', 'n_estimators': 100, 'warm_start': False}
    Best score -> 0.9378787878787879
Si usamos el dataset de test, obtenemos el siguiente resultado:
    score = 0.9736842105263158
Tiempo medio en ejecutarse el método: 0.12454408937030369 +- 0.009300579489462965s
Tiempo en ejecutarse la búsqueda 3.129668951034546s, (0.0521611491839091 min)
\begin{tabular}{llrrr}
\toprule
{} &     Dataset &  TimeMethod &  TimeSearch &     Score \\
\midrule
0 &  Ionosphere &    0.148219 &    3.596214 &  0.965909 \\
1 &    Diabetes &    0.111805 &    2.726361 &  0.739583 \\
2 &     Vehicle &    0.157361 &    3.815310 &  0.971698 \\
3 &       Vowel &    0.174096 &    4.208067 &  0.987854 \\
4 &        Iris &    0.124544 &    3.129669 &  0.973684 \\
\bottomrule
\end{tabular}



In [99]:
compareDF = pd.DataFrame()
compareDF['SVM'] = ScoreSVM
compareDF['Tree'] = ScoreTree
compareDF['BaggingSVM'] = ScoreBaggingSVM
compareDF['BaggingTree'] = ScoreBaggingTree
compareDF['Boosting1_SVM'] = ScoreBoosting1_SVM
compareDF['Boosting1_Tree'] = ScoreBoosting1_Tree
compareDF['Boosting2_Tree'] = ScoreBoosting2_Tree
compareDF['GradBoost'] = ScoreGradBoost
compareDF.to_csv("performanceGridSearch.csv", index=False)
print(compareDF)
print(compareDF.to_latex())

        SVM      Tree  BaggingSVM  BaggingTree  Boosting1_SVM  Boosting1_Tree  \
0  0.920455  0.920455    0.897727     0.920455       0.965909        0.931818   
1  0.755208  0.723958    0.661458     0.729167       0.661458        0.729167   
2  0.966981  0.957547    0.731132     0.971698       0.750000        0.938679   
3  0.971660  0.971660    0.906883     0.995951       1.000000        0.991903   
4  1.000000  0.921053    0.736842     0.973684       0.947368        0.947368   

   Boosting2_Tree  GradBoost  
0        0.852273   0.965909  
1        0.734375   0.739583  
2        0.929245   0.971698  
3        0.991903   0.987854  
4        0.868421   0.973684  
\begin{tabular}{lrrrrrrrr}
\toprule
{} &       SVM &      Tree &  BaggingSVM &  BaggingTree &  Boosting1\_SVM &  Boosting1\_Tree &  Boosting2\_Tree &  GradBoost \\
\midrule
0 &  0.920455 &  0.920455 &    0.897727 &     0.920455 &       0.965909 &        0.931818 &        0.852273 &   0.965909 \\
1 &  0.755208 &  0.723958 &   

# Iman Davenport test

In [106]:
import numpy as np
from scipy.stats import chi2

def compute_imon_davenport_statistic(performance_matrix):
  # Compute the ranks of the model performance on each dataset
  ranks = np.apply_along_axis(lambda x: len(x) - np.argsort(np.argsort(x)), 1, performance_matrix)

  # Sum the ranks for each model across all datasets
  rank_sums = np.sum(ranks, axis=0)

  # Compute the iman Davenport statistic
  iman_davenport_statistic = (np.max(rank_sums) - np.min(rank_sums)) / performance_matrix.shape[1]

  return iman_davenport_statistic

def compute_p_value(imon_davenport_statistic, num_models, num_datasets):
  # Compute the degrees of freedom for the Imon Davenport test
  df = num_models - 1

  # Compute the p-value using the chi-squared distribution
  p_value = 1 - chi2.cdf(imon_davenport_statistic, df)

  return p_value

def imon_davenport_test(performance_matrix, significance_level):
  # Compute the Imon Davenport statistic and p-value
  imon_davenport_statistic = compute_imon_davenport_statistic(performance_matrix)
  p_value = compute_p_value(imon_davenport_statistic, performance_matrix.shape[0], performance_matrix.shape[1])

  # Determine whether the difference in performance between the models is statistically significant
  if p_value < significance_level:
    print(f"The difference in performance between the models is statistically significant (p = {p_value:.3f})")
  else:
    print(f"The difference in performance between the models is not statistically significant (p = {p_value:.3f})")
  return p_value

In [109]:
performance_matrix = pd.read_csv("performanceGridSearch.csv")
# Run the Imon Davenport test
p_value = imon_davenport_test(performance_matrix, 0.05)

The difference in performance between the models is not statistically significant (p = 0.478)


# Wilcoxon Test

In [102]:
# If there are significant differences between the models, then apply the wilcoxon test to determine which models are significantly different
if p_value < 0.05:
  # Compute the pairwise differences between the models
  pairwise_differences = np.apply_along_axis(lambda x: x - x[:, None], 1, performance_matrix)

  # Compute the p-values for the pairwise differences using the Wilcoxon signed-rank test
  p_values = np.apply_along_axis(lambda x: wilcoxon(x, zero_method="wilcox")[1], 1, pairwise_differences)

  # Compute the Bonferroni correction
  bonferroni_correction = 0.05 / (p_values.shape[0] * (p_values.shape[0] - 1) / 2)

  # Determine which models are significantly different
  significant_differences = np.argwhere(p_values < bonferroni_correction)

  # Print the significant differences
  for difference in significant_differences:
    print(f"Model {difference[0]} is significantly different from model {difference[1]}")